In [2]:
import numpy as np

### Loading Image

In [245]:
# Load P1 Image
def load_image(image_url):
    header = []
    with open(image_url, 'r') as f:
        for l in range(3):
            header.append(f.readline())
        y,x = [int(n) for n in header[2].split()]
        pixels = np.zeros((x, y)) # (7,24)
        for j in range(x):
            for k in range(y):
                pixel = f.read(1)
                if pixel in ['\n', ' ']:
                    pixel = f.read(1)
                pixels[j][k] = pixel
        return header, pixels

In [247]:
img = load_image('arquivos/lorem_s12_c02_just.pbm')
img

(['P1\n', '# CREATOR: GIMP PNM Filter Version 1.1\n', '1653 2338\n'],
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]))

In [219]:
# DRAFT
# img = load_image('teste.pbm')
# img
# f = open('arquivos/lorem_s12_c02_just.pbm', 'r')
f = open('teste.pbm', 'r')
f.readlines(), open('fig_test.pbm', 'r').readlines()


(['P1\n',
  '# PBM example \n',
  '24 7\n',
  '000000000000000000000000\n',
  '011110011110011110011110\n',
  '010000010000010000010010\n',
  '011100011100011100011110\n',
  '010000010000010000010000\n',
  '010000011110011110010000\n',
  '000000000000000000000000\n'],
 ['P1\n',
  '# PBM example \n',
  '24 7\n',
  '000000000000000000000000\n',
  '011110011110011110011110\n',
  '010000010000010000010010\n',
  '011100011100011100011110\n',
  '010000010000010000010000\n',
  '010000011110011110010000\n',
  '000000000000000000000000\n'])

### Store image w/ transformations

In [242]:
def store_image(image_url, img_data, transform_func=None, threshold=None):
    header, pixels = img_data
    with open(image_url, 'w') as f:
        print("header", header)
        [f.write(l) for l in header]
        y,x = [int(n) for n in header[2].split()]
        print(x,y)
        
#         LMax = int(header[3]) + 1
#         if transform_func == 'equalize':
#             equalize([pixels, LMax])
        for j in range(x):
            for k in range(y):
                # 3th - Negative filter
#                 if transform_func == 'negative':
#                     pixels[j][k] = (LMax - 1) - pixels[j][k]
#                 # 3th - Thresholding filter
#                 if transform_func == 'threshold':
#                     pixels[j][k] = int(header[3]) if pixels[j][k] >= threshold else 0                    
#                 print(pixels[j][k])
                f.write(str(int(pixels[j][k])))
            f.write('\n')
        return 0


In [248]:
store_image('teste.pbm', img)

header ['P1\n', '# CREATOR: GIMP PNM Filter Version 1.1\n', '1653 2338\n']
2338 1653


0

In [249]:
load_image('teste.pbm')

(['P1\n', '# CREATOR: GIMP PNM Filter Version 1.1\n', '1653 2338\n'],
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]))

In [ ]:
# def level_transform(pixels, LMax, transformations=[]):
#     if len(transformations) == 0: return
#     for transform_func in transformations:
#         if transform_func == 'negative': 
#                 pixels[j][k] = ((int(header[3])) - pixels[j][k]) # header[3] is already L-1
#         if transform_func == 'threshold':
#                 pixels[j][k] = int(header[3]) if pixels[j][k] >= threshold else 0

In [ ]:
# Bring down the transform functions to a proper function like that:
"""
    transform=None # expects array of object strings
    provide it as below:
    
    def level_transform(transformations=[]):
        for transform_func in transformations:
            if transform_func == 'negative': 
                    pixels[j][k] = ((int(header[3])) - pixels[j][k]) # header[3] is already L-1
            if transform_func == 'threshold':
                    pixels[j][k] = int(header[3]) if pixels[j][k] >= threshold else 0
"""

## Histograma é dado por
$ h(r_k) = n_k $, onde $r_k$ é a tonalidade da imagem com k = 0,1,2...L-1 e $n_k$ é o número de pixels na tonalidade k

In [ ]:
# Histogram of the picture
def h(img_data):
    LMax = int(header[3]) + 1
    H = np.zeros(LMax).astype('int64')
    for x in range(img_data.shape[0]):
        for y in range(img_data.shape[1]):
            H[img_data[x][y]] += 1
    return H

### Normalização de histograma é dada por
$p(r_k) = n_k/M.N$, onde M e N representam o número total de pixels da imagem

In [ ]:
def normalize(histogram, resolution):
    M, N = resolution
    return histogram / (M*N)

In [ ]:
### Cummulative summation of the probabilities

# P.sum() # Summation of the K_(L-1) probabilities
def cumsum(P):
    C = np.zeros_like(P)
    for i in range(C.size):
        cumsum = 0
        for k in range(i+1):
            cumsum += P[k]
        C[i] = cumsum
    return C

In [ ]:
def equalize(img_data=[]):
    if len(img_data) != 2: return
    
    pixels, LMax = img_data
    resolution = pixels.shape
    histogram = h(pixels)
    # Normalize
    P = normalize(histogram, resolution)
    # Sum Probabilities
    C = cumsum(P)
    # Map new indexes
    eq = LMax - 1
    equalized_histogram = np.array(
        np.round(eq * C)
    ).astype('int64')
    # equalize each pixel
    for i in range(pixels.shape[0]):
        for j in range(pixels.shape[1]):
            pixels[i][j] = equalized_histogram[pixels[i][j]]

### Filtro da mediana

In [ ]:
import numpy as np
# criar matriz 3x3 
median = np.zeros((3,3))
# median
# carregue a imagem e itere
#p1 loader
# header, pixels = load_image('arquivos/lorem_s12_c02.pbm')
header, pixels = load_image('./fig_test.pbm')

url = 'imagem.pbm'

store_image(url)
#read with p2 loader
# load_image('../Figuras/barco.pgm')